## Numerical Analysis - Fall semester 2024

# Serie 09 - Numerical integration and linear systems

Package imports.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import time

<hr style="clear:both">

### Error indicators for linear systems

We consider the Vandermonde matrix

$$
A = \begin{pmatrix}
  1 & t_1 & t_1^2 & \cdots & t_1^{n-1}\\
  1 & t_2 & t_2^2 & \cdots & t_2^{n-1} \\
  \vdots & \vdots & \vdots & \ddots  & \vdots \\
  1 & t_n & t_n^2 & \cdots & t_n^{n-1} 
\end{pmatrix} \in \mathbb{R}^{n\times n},
$$

where the points $t_1, \ldots, t_n$ are equispaced in the interval $[0, 1]$. For a set of points `t`, the Vandermonde matrix can be generated with `A = np.vander(t, increasing=True)`

We want to solve the linear system $A\mathbf{x}=\mathbf{b}$ for $\mathbf{x}$ where
$$
\mathbf{b} = \begin{pmatrix}
  1+t_1^2 \\
  1+t_2^2 \\
  \vdots \\
  1+t_n^2 
\end{pmatrix}  \in \mathbb{R}^{n}.
$$
Clearly, the exact solution is $\mathbf{x}=(1,0,1,0\ldots,0)^\top \in \mathbb{R}^{n}$.

<div class="alert alert-success">
    
**Exercise 1:** For $n=4$, solve the linear system numerically using the command `np.linalg.solve`. Let $\mathbf{x_c}$ be the obtained solution. Compute the relative error $\varepsilon=\|\mathbf{x_c}-\mathbf{x}\| / \|\mathbf{x}\|$. 

*Hint:* The Euclidean norm of a vector can be computed with the NumPy command `np.linalg.norm`.
</div>


In [ ]:
### BEGIN SOLUTION
n = 4
t = np.linspace(0, 1, n)
A = np.vander(t, increasing=True)
b = 1 + t ** 2

x_c = np.linalg.solve(A, b)

x_ex = np.zeros(n)
x_ex[[0, 2]] = 1
err = np.linalg.norm(x_c - x_ex) / np.linalg.norm(x_ex)

print(r"relative error of obtained solution: ε = {:.4e}".format(err))
### END SOLUTION

An upper bound of the relative error $\varepsilon$ is given by
$$
\eta = \kappa(A)~\text{eps}
$$
where $\kappa(A)$ is the condition number of the matrix $A$
(which can be estimated using the command `np.linalg.cond`) and $\text{eps}$ the rounding unit of the floating point representation (which is defined in the variable `sys.float_info.epsilon`). 

<div class="alert alert-success">
    
**Exercise 2:** For $n=4$, compute $\eta$ and check if this is an upper bound on the relative error $\varepsilon$
computed in the previous exercise.
</div>

In [ ]:
### BEGIN SOLUTION
eps = sys.float_info.epsilon
cond_A = np.linalg.cond(A)
eta = cond_A * eps

print(r"upper bound for relative error of obtained solution: η = {:.4e}".format(eta))
# We see that the bound η is an upper bound on the true relative error ε.
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 3:** For the same linear system but for matrix sizes $n=4, 6, 8, \ldots, 20$, visualize the relative error $\varepsilon$, the upper bound $\eta$, and the normalized residual $r = \|\mathbf{b}-A\mathbf{x_c}\|/\|\mathbf{b}\|$ in terms of $n$, in two graphs, one in logarithmic scale (`plt.loglog`) and one in semi-logarithmic scale (`plt.semilogy`). What type of growth do we see for the  error $\varepsilon$? Is the residual $r$ a good indicator of the error $\varepsilon$?
</div>

In [ ]:
### BEGIN SOLUTION
n_list = 2 * np.arange(2, 11)

eta_list = []
err_list = []
res_list = []

for n in n_list:

    x = np.linspace(0, 1, n)
    A = np.vander(x, increasing="True")   
    b = 1 + x ** 2
    x_ex = np.zeros(n)
    x_ex[[0, 2]] = 1
    
    cond_A = np.linalg.cond(A) 
    x_c = np.linalg.solve(A, b)
    
    eta_list.append(cond_A * eps)
    res_list.append(np.linalg.norm(b - A @ x_c) / np.linalg.norm(b))
    err_list.append(np.linalg.norm(x_c - x_ex) / np.linalg.norm(x_ex))

plt.loglog(n_list, eta_list, label=r"upper bound $\eta$")
plt.loglog(n_list, err_list, label=r"relative error $\varepsilon$")
plt.loglog(n_list, res_list, color="black", linestyle="--", label=r"residual $r$")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.xlabel(r"matrix size $n$")
plt.show()

plt.semilogy(n_list, eta_list, label=r"upper bound $\eta$")
plt.semilogy(n_list, err_list, label=r"relative error $\varepsilon$")
plt.semilogy(n_list, res_list, color="black", linestyle="--", label=r"residual $r$")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.xlabel(r"matrix size $n$")
plt.show()

# We notice that the bound η is quite accurate. Moreover, the relative
# error as well as the upper bound grow linearly in the graph in semi-logarithmic
# scale, which shows that the growth of the error is exponential:
# ε ≈ exp(αn) for some α > 0.

# However, the residual is not at all a good indicator of the error as it is always
# on the order of 10⁻¹⁵ while the true relative error becomes quite large, of
# the order of a few percents for n = 20. This is due to the fact that the matrix
# is ill-conditioned.
### END SOLUTION

We now consider a different matrix
$$
B = \begin{pmatrix} 2 & -1 & & &  \\ -1 & 2 & \ddots & &  \\ & \ddots
& \ddots & \ddots &  \\ & &\ddots & \ddots &  -1 \\ &&&-1&2 \end{pmatrix} \in \mathbb{R}^{n \times n}
$$
which can be generated with the command `B = np.diag(2 * np.ones(n), 0)-np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)`.

We want to solve the linear system $B \mathbf{y}=\mathbf{c}$ where $\mathbf{c}=(2,2,\cdots,2)^\top \in \mathbb{R}^{n}$. The exact solution in this case is
$$
\mathbf{y} =
\begin{pmatrix}
1\cdot(n) \\
2\cdot(n-1)\\
3\cdot(n-2)\\
\vdots \\
n\cdot 1 
\end{pmatrix} \in \mathbb{R}^{n}.
$$

<div class="alert alert-success">
    
**Exercise 4:** For $n=5, 10, \ldots, 100$, repeat the previous exercise for the matrix $B$. Comment on the result.
</div>

In [ ]:
### BEGIN SOLUTION
n_list = 5 * np.arange(1, 21)

eta_list = []
err_list = []
res_list = []

for n in n_list:

    y = np.linspace(0, 1, n)
    B = np.diag(2 * np.ones(n), 0) - np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)
    c = 2 * np.ones(n)
    y_ex = np.arange(1, n + 1) * np.arange(n, 0, step=-1)
    
    cond_B = np.linalg.cond(B) 
    y_c = np.linalg.solve(B, c)
    
    eta_list.append(cond_B * eps)
    res_list.append(np.linalg.norm(c - B @ y_c) / np.linalg.norm(c))
    err_list.append(np.linalg.norm(y_c - y_ex) / np.linalg.norm(y_ex))

plt.loglog(n_list, eta_list, label=r"upper bound $\eta$")
plt.loglog(n_list, err_list, label=r"relative error $\varepsilon$")
plt.loglog(n_list, res_list, color="black", linestyle="--", label=r"residual $r$")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.xlabel(r"matrix size $n$")
plt.show()

plt.semilogy(n_list, eta_list, label=r"upper bound $\eta$")
plt.semilogy(n_list, err_list, label=r"relative error $\varepsilon$")
plt.semilogy(n_list, res_list, color="black", linestyle="--", label=r"residual $r$")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.xlabel(r"matrix size $n$")
plt.show()

# In this case, the error as well as the upper bound increase much more slowly. We
# can conclude from the upper graph that the relative error increases as ε ≈ n².
# Moreover, the normalized residual provides this time a reasonable bound on the
# relative error, better than the bound η based on the condition number of the matrix.
### END SOLUTION

<hr style="clear:both">

### Richardson's methods

<div class="alert alert-success">
    
**Exercise 5:** Complete the below implementation of the Richardson's method with stopping criterion (see Algorithm 5.2 in the lecture notes).
</div>

In [ ]:
def richardson(A, b, P, x_0, n_max, tol):
    x = x_0
    res = b - A @ x
    niter = 0
    while np.linalg.norm(res) > tol * np.linalg.norm(b) and niter < n_max:
        z = np.linalg.solve(P, res)
        ### BEGIN SOLUTION
        x = x + z
        res = b - A @ x
        niter = niter + 1
        ### END SOLUTION
    return x, res, niter

We now use
$$
A=
\begin{pmatrix}
  1 & 0.5 & 0 \\
  0 & 2 & 1 \\
  -1 & 1 & 1
\end{pmatrix}, \quad \mathbf{b}=
\begin{pmatrix}
  2 \\
  5 \\
  2
\end{pmatrix}.
$$

<div class="alert alert-success">
    
**Exercise 6:** Implement the Jacobi method and iteratively solve the linear system $A\mathbf{x}=\mathbf{b}$ using the parameters $\mathbf{x}^{(0)}=\mathbf{0}$, $n_{\max}=10^4$, and $tol=10^{-5}$.

*Hint:* The Jacobi method is a Richardson's method for a specific choice of $P$ (see Chapter 5.2 in the lecture notes).
</div>

In [ ]:
def jacobi(A, b, x_0, n_max, tol):
    ### BEGIN SOLUTION
    P = np.diag(np.diag(A))
    x, res, niter = richardson(A, b, P, x_0, n_max, tol)
    ### END SOLUTION
    return x, res, niter

### BEGIN SOLUTION
A = np.array([[1, 0.5, 0], [0, 2, 1], [-1, 1, 1]])
b = np.array([2, 5, 2])
x_0 = np.zeros(3)
n_max = 1e4
tol = 1e-5

x, res, niter = jacobi(A, b, x_0, n_max, tol)
print(r"obtained solution: x = {}".format(x))
print(r"residual of solution: res = {}".format(res))
print(r"number of iterations: niter = {}".format(niter))
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 7:** Repeat Exercise 6 using the Gauss-Seidel method.

*Hint:* The NumPy function `np.tril` can be used to extract the lower triangular part of a matrix.
</div>

In [ ]:
def gauss_seidel(A, b, x_0, n_max, tol):
    ### BEGIN SOLUTION
    P = np.tril(A)
    return richardson(A, b, P, x_0, n_max, tol)
    ### END SOLUTION

### BEGIN SOLUTION
A = np.array([[1, 0.5, 0], [0, 2, 1], [-1, 1, 1]])
b = np.array([2, 5, 2])
x_0 = np.zeros(3)
n_max = 1e4
tol = 1e-5

P_gauss_seidel = np.tril(A)
x, res, niter = gauss_seidel(A, b, x_0, n_max, tol)
print(r"obtained solution: x = {}".format(x))
print(r"residual of solution: res = {}".format(res))
print(r"number of iterations: niter = {}".format(niter))
### END SOLUTION

Let us now consider the new linear system $L\mathbf{y}=\mathbf{f}$ for the tridiagonal matrix $$
  L=\begin{pmatrix} 2.5 & -1 & & &  \\ -1 & 2.5 & -1 & &  \\ & \ddots
    & \ddots & \ddots &  \\ & &-1 & 2.5 &  -1 \\ &&&-1&2.5 \end{pmatrix} \in \mathbb{R}^{n \times n}, \quad \mathbf{f}=
\begin{pmatrix}
  1.5 \\
  0.5 \\
  0.5 \\
  \vdots \\
  0.5 \\
  1.5
\end{pmatrix} \in \mathbb{R}^{n}.
$$
which you can generate with the command `L = np.diag(2.5 * np.ones(n), 0) - np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)`.

<div class="alert alert-success">
    
**Exercise 8:** For $n=4$, solve the linear system with the Jacobi and Gauss-Seidel methods you've implemented in Exercises 6 and 7. Use the initial data $\mathbf{y}^{(0)}=\mathbf{0}$, a tolerance of $10^{-10}$, and a maximum number of iterations $10^8$. Note down the computing time as well as the number of iterations. What can we say about the convergence of the two methods?  Which one converges faster?
*Hint:* You can measure the time (in seconds) an operation takes with
```python
start_time = time.time()
# Some operation ...
end_time = time.time()
run_time = end_time - start_time
```
</div>

In [ ]:
# Assemble the matrix and right-hand side
n = 4
L = np.diag(2.5 * np.ones(n), 0) - np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)
f = 0.5 * np.ones(n)
f[[0, -1]] = 1.5

### BEGIN SOLUTION
y_0 = np.zeros(n)
tol = 1e-10
n_max = 1e8
  
start_time = time.time()
y, res, niter = jacobi(L, f, y_0, n_max, tol)
end_time = time.time()
run_time = end_time - start_time
print("Jacobi:\n" + 7*"-")
print(r"obtained solution: x = {}".format(y))
print(r"residual of solution: res = {}".format(res))
print(r"number of iterations: niter = {}".format(niter))
print(r"computing time: t = {}".format(run_time))

start_time = time.time()
y, res, niter = gauss_seidel(L, f, y_0, n_max, tol)
end_time = time.time()
run_time = end_time - start_time
print("\nGauss-Seidel:\n" + 13*"-")
print(r"obtained solution: x = {}".format(y))
print(r"residual of solution: res = {}".format(res))
print(r"number of iterations: niter = {}".format(niter))
print(r"computing time: t = {}".format(run_time))

# We notice here that both methods converge. The Jacobi method needs 53 iterations
# and approximately while the Gauss-Seidel method takes 27 iterations. The Gauss-Seidel
# method is approximately twice as fast, since we have ρ(B_GS) = ρ(B_J)² < ρ(B_J) for the
# iteration matrices B_J (Jacobi method) and B_GS (Gauss-Seidel method). See Theorem 5.1
# in the lecture notes. 
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 9:** Solve the linear system $L \mathbf{y} = \mathbf{f}$ from above for $n=4,8,12,...,200$ with initial iterate $\mathbf{y}^{(0)}=\mathbf{0}$, tolerance $10^{-10}$, and maximum number of iterations $10^8$. How does the number of iterations in terms of $n$ behave? Knowing that the exact solution of the system is $\mathbf{y}=(1,...,1)^\top$ and denoting the approximate solution as $\mathbf{y}_c$, plot the relative error $\|\mathbf{y}_c-\mathbf{y}\|/\|\mathbf{y}\|$ and the normalized residual $\|\mathbf{f} - L \mathbf{y}_c\|/\|\mathbf{f}\|$ in terms of $n$ in a graph in logarithmic scale. Is the residual a good estimator of the error? How does the condition number of $L$ change in terms of $n$?
</div>

In [ ]:
### BEGIN SOLUTION
tol = 1e-10
n_max = 1e8

err_list_jacobi = []
res_list_jacobi = []
niter_list_jacobi = []

err_list_gaussseidel = []
res_list_gaussseidel = []
niter_list_gaussseidel = []

cond_list = []

n_list = 4 * np.arange(1, 51)
for n in n_list:

    L = np.diag(2.5 * np.ones(n), 0) - np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)
    f = 0.5 * np.ones(n)
    f[[0, -1]] = 1.5
    y_0 = np.zeros(n)
    y_ex = np.ones(n)
    
    y_c_jacobi, _, niter_jacobi = jacobi(L, f, y_0, n_max, tol)
    y_c_gaussseidel, _, niter_gaussseidel = gauss_seidel(L, f, y_0, n_max, tol)
    
    cond_list.append(np.linalg.cond(L))
    
    res_list_jacobi.append(np.linalg.norm(f - L @ y_c_jacobi) / np.linalg.norm(f))
    err_list_jacobi.append(np.linalg.norm(y_c_jacobi - y_ex) / np.linalg.norm(y_ex))
    niter_list_jacobi.append(niter_jacobi)
                             
    res_list_gaussseidel.append(np.linalg.norm(f - L @ y_c_gaussseidel) / np.linalg.norm(f))
    err_list_gaussseidel.append(np.linalg.norm(y_c_gaussseidel - y_ex) / np.linalg.norm(y_ex))
    niter_list_gaussseidel.append(niter_gaussseidel)

plt.loglog(n_list, niter_list_jacobi, label="Jacobi")
plt.loglog(n_list, niter_list_gaussseidel, label="Gauss-Seidel")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.ylabel(r"number of iterations")
plt.xlabel(r"matrix size $n$")
plt.show()

# The number of iterations increases and then stabilizes to 103 for
# Jacobi's method and to 57 for Gauss-Seidel.

plt.title("Jacobi method")
plt.loglog(n_list, err_list_jacobi, label=r"relative error $\varepsilon$")
plt.loglog(n_list, res_list_jacobi, label=r"residual $r$")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.xlabel(r"matrix size $n$")
plt.show()

plt.title("Gauss-Seidel method")
plt.loglog(n_list, err_list_gaussseidel, label=r"relative error $\varepsilon$")
plt.loglog(n_list, res_list_gaussseidel, label=r"residual $r$")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.xlabel(r"matrix size $n$")
plt.show()

plt.loglog(n_list, cond_list)
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.ylabel(r"condition number $\kappa(L)$")
plt.xlabel(r"matrix size $n$")
plt.show()

# We notice that for the matrix L, the residual is a good estimator of the error.
# We can explain this using the fact that the matrix L is well conditioned for
# all the values of n (see condition number graph). For example, for n=200,
# the condition number of the matrix is approximately 9. The error and the
# residual r_k at the k-th iteration are linked by the relation
# 
#     ||y - y_k|| / ||y|| <= κ(L) * ||r_k|| / ||f||
#
# Therefore, the residual is a good estimator of the error when the conditioning
# of the matrix is close to 1.

### END SOLUTION

<div class="alert alert-success">
    
**Exercise 10:** For $n=4, 8, 12, \dots, 48$, repeat the previous exercise on the matrix
$$
  L=\begin{pmatrix} 2 & -1 & & &  \\ -1 & 2 & -1 & &  \\ & \ddots
    & \ddots & \ddots &  \\ & &-1 & 2 &  -1 \\ &&&-1&2 \end{pmatrix} \in \mathbb{R}^{n \times n}
$$
and the right-hand side $\mathbf{f} = (1, 0, \dots, 0, 1)^{\top}$, such that the exact solution will still be the same.
</div>

In [ ]:
### BEGIN SOLUTION
tol = 1e-10
n_max = 1e8

err_list_jacobi = []
res_list_jacobi = []
niter_list_jacobi = []

err_list_gaussseidel = []
res_list_gaussseidel = []
niter_list_gaussseidel = []

cond_list = []

n_list = 4 * np.arange(1, 13)
for n in n_list:

    L = np.diag(2 * np.ones(n), 0) - np.diag(np.ones(n - 1), 1) - np.diag(np.ones(n - 1), -1)
    f = np.zeros(n)
    f[[0, -1]] = 1
    y_0 = np.zeros(n)
    y_ex = np.ones(n)
    
    y_c_jacobi, _, niter_jacobi = jacobi(L, f, y_0, n_max, tol)
    y_c_gaussseidel, _, niter_gaussseidel = gauss_seidel(L, f, y_0, n_max, tol)
    
    cond_list.append(np.linalg.cond(L))
    
    res_list_jacobi.append(np.linalg.norm(f - L @ y_c_jacobi) / np.linalg.norm(f))
    err_list_jacobi.append(np.linalg.norm(y_c_jacobi - y_ex) / np.linalg.norm(y_ex))
    niter_list_jacobi.append(niter_jacobi)
                             
    res_list_gaussseidel.append(np.linalg.norm(f - L @ y_c_gaussseidel) / np.linalg.norm(f))
    err_list_gaussseidel.append(np.linalg.norm(y_c_gaussseidel - y_ex) / np.linalg.norm(y_ex))
    niter_list_gaussseidel.append(niter_gaussseidel)

plt.loglog(n_list, niter_list_jacobi, label="Jacobi")
plt.loglog(n_list, niter_list_gaussseidel, label="Gauss-Seidel")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.ylabel(r"number of iterations")
plt.xlabel(r"matrix size $n$")
plt.show()

plt.title("Jacobi method")
plt.loglog(n_list, err_list_jacobi, label=r"relative error $\varepsilon$")
plt.loglog(n_list, res_list_jacobi, label=r"residual $r$")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.xlabel(r"matrix size $n$")
plt.show()

plt.title("Gauss-Seidel method")
plt.loglog(n_list, err_list_gaussseidel, label=r"relative error $\varepsilon$")
plt.loglog(n_list, res_list_gaussseidel, label=r"residual $r$")
plt.legend()
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.xlabel(r"matrix size $n$")
plt.show()

plt.loglog(n_list, cond_list)
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.ylabel(r"condition number $\kappa(A)$")
plt.xlabel(r"matrix size $n$")
plt.show()

# The necessary number of iterations increases when n increases.
# This is due to the fact that the norm of the iteration matrices B_J and B_GS
# gets closer and closer to 1 when n increases.

# Moreover, we notice that in this case the residual is not at all a good
# estimator of the error. This is due to the fact that the conditioning of
# the matrix increases more and more when n increases, as we can see on the
# fourth plot.

### END SOLUTION

The eigenvalues of a tridiagonal matrix of the form

$$
L = \begin{bmatrix} a & b & & &  \\ c & a & b & &  \\ & \ddots & \ddots & \ddots &  \\ & & c & a & b \\ &&&c&a \end{bmatrix} \in\mathbb{R}^{n\times n}
$$
are $\lambda_i(L)=a+2\sqrt{bc}\cos\left(\frac{\pi i}{n+1}\right)$, $i=1,...,n$. 

<div class="alert alert-success">
    
**Exercise 11 (Theoretical):** Give the condition number $\kappa(L)$ of the tridiagonal matrix $L$ explicitly in the case where $b=c$ and $|b|\leq a/2$, i.e. for positive semi-definite $L$. Can this explain the numerical results obtained in the two previous exercises?

*Hint:* For a symmetric matrix $L$, the condition number is given by $\kappa(L) = \lambda_{\max}(L) / \lambda_{\min}(L)$.
</div>

=== BEGIN MARK SCHEME ===

The condition number of a matrix $L$, which is symmetric and positive definite, is given by
$\kappa(L)=\frac{\lambda_{\max}(L)}{\lambda_{\min}(L)}$. For the considered tridiagonal
matrix $L$, the eigenvalues are given by $\lambda_i(L)=a+2|b|\cos(\pi i/(n+1))$ where $|\cdot|$
is the absolute value. As the maximum and the minimum of $\cos(\pi i/(n+1))$ are reached in
$i=1$ and $i=n$, respectively, the maximal an minimal eigenvalues are given by

\begin{align*}
\lambda_{\max}(L)&=a+2|b|\cos\left(\frac{\pi}{n+1}\right) \\
\lambda_{\min}(L)&=a+2|b|\cos\left(\frac{\pi n}{n+1}\right)=a-2|b|\cos\left(\frac{\pi}{n+1}\right),
\end{align*}

and the condition number of the matrix is therefore

$$
\kappa(L)=\frac{a+2|b|\cos\left(\frac{\pi}{n+1}\right)}{a-2|b|\cos\left(\frac{\pi}{n+1}\right)}.
$$

With $a=2.5$ and $b=-1$ we get for $n=200$ a condition number of $\kappa(L)\approx 8.995$ whereas for $a=2$ and $b=-1$, we have $\kappa(L)\approx 16373$ which is consistent with the numerical results obtained above.

=== END MARK SCHEME ===

<hr style="clear:both">

### Condition number of simple matrices

Given are the matrices

\begin{align*}
  D=
  \begin{pmatrix}
    d_1 & 0 \\
    0   & d_2
  \end{pmatrix}, \quad 
  R=
  \begin{pmatrix}
    \cos(\theta) & -\sin(\theta) \\
    \sin(\theta) & \cos(\theta)
  \end{pmatrix}, \quad
  L=
  \begin{pmatrix}
    1 & 0 \\
    a & 1
  \end{pmatrix},
\end{align*}

where $d_1>d_2>0$, $0 < \theta < 2 \pi$ and $a=1000$.

<div class="alert alert-success">
    
**Exercise 12 (Theoretical):** Compute the condition number of the matrices $D$, $R$, and $L$.

*Hint:* You can also use Python to make an educated guess.
</div>

=== BEGIN MARK SCHEME ===

The condition number of a matrix $A$ is defined by

$$
\kappa(A)=\sqrt{\frac{\lambda_{\max}(A^\top A)}{\lambda_{\min}(A^\top A)}},
$$

where $\lambda_{\max}(A)$ is the largest eigenvalue of $A$ and $\lambda_{\min}(A)$ is the smallest eigenvalue of $A$. To solve
the exercise, we have to compute the eigenvalues of the matrices $D^\top D$, $R^\top R$ and $L^\top L$.

**Matrix $D$**

$$
D^\top D = 
\begin{pmatrix}
  d_1 & 0 \\
  0   & d_2 
\end{pmatrix}
\begin{pmatrix}
  d_1 & 0 \\
  0   & d_2 
\end{pmatrix}
=
\begin{pmatrix}
  d_1^2 & 0 \\
  0   & d_2^2 
\end{pmatrix}
$$

and therefore $\lambda_{\max}(D^\top D)=d_1^2, \lambda_{\min}(D^\top D)=d_2^2$ and $\displaystyle \kappa(D)=\frac{d_1}{d_2}$.

**Matrix $R$**

$$
R^\top R=
\begin{pmatrix}
  \cos(\theta) & \sin(\theta) \\
  -\sin(\theta) & \cos(\theta)
\end{pmatrix}
\begin{pmatrix}
  \cos(\theta) & -\sin(\theta) \\
  \sin(\theta) & \cos(\theta)
\end{pmatrix}=
\begin{pmatrix}
  1 & 0 \\
  0 & 1
\end{pmatrix}
$$

and therefore $\lambda_{\max}(R^\top R)=\lambda_{\min}(R^\top R)=1$ and $\displaystyle \kappa(R)=1$.

**Matrix $L$**

$$
L^\top L=
\begin{pmatrix}
  1 & a \\
  0 & 1
\end{pmatrix}
\begin{pmatrix}
  1 & 0 \\
  a & 1
\end{pmatrix} = 
\begin{pmatrix}
  a^2+1 & a \\
  a & 1
\end{pmatrix}.
$$

The eigenvalues of $L^\top L$ solve $\det(L^\top L-\lambda I)=0,$ namely

$$
0 = \det
\begin{pmatrix}
  a^2+1-\lambda & a \\
  a & 1-\lambda
\end{pmatrix}
= (a^2+1-\lambda)(1-\lambda) - a^2
= \lambda^2 -(2+a^2)\lambda + 1,
$$

whence we get

$$
\lambda_{\max}(L^\top L) = \frac{2+a^2+a\sqrt{a^2 + 4}}{2} \approx 10^6, \quad
\lambda_{\min}(L^\top L) = \frac{2+a^2-a\sqrt{a^2 + 4}}{2} \approx 10^{-6}
$$

and therefore $\kappa(L) \approx 10^6$.

=== END MARK SCHEME ===

<hr style="clear:both">

### Convergence of Jacobi's method

We consider the linear system $A \mathbf{x}=\mathbf{b}$ with

$$
  A = \begin{pmatrix}
        1 & 0 & \alpha \\
        \beta & 1 & 0 \\
        0 & \gamma & 1 
       \end{pmatrix}
      , \qquad \mathbf{b}=  \begin{pmatrix} 1\\ 3\\ 2\end{pmatrix}
$$
where $\alpha, \beta, \gamma \in \mathbb{R}$ are given.

<div class="alert alert-success">
    
**Exercise 13 (Theoretical):** Write Jacobi's method to solve the linear system $A\bf{x}=\bf{b}$. That is, express the new iterates $x_1^{(k+1)}$, $x_2^{(k+1)}$, and $x_3^{(k+1)}$ in terms of the previous iterates $x_1^{(k)}$, $x_2^{(k)}$, and $x_3^{(k)}$.
</div>

=== BEGIN MARK SCHEME ===

$$
\left\{
\begin{array}{l}
x_1^{(k+1)}=\frac{1}{a_{11}}\, \left( b_1 - a_{12}\, x_2^{(k)} - a_{13}\, x_3^{(k)} \right)= 1 - \alpha\, x_3^{(k)} \\[5mm]
x_2^{(k+1)}=\frac{1}{a_{22}}\, \left( b_2 - a_{21}\, x_1^{(k)} - a_{23}\, x_3^{(k)} \right)= 3 - \beta\, x_1^{(k)} \\[5mm]
x_3^{(k+1)}=\frac{1}{a_{33}}\, \left( b_3 - a_{31}\, x_1^{(k)} - a_{32}\, x_2^{(k)} \right)= 2 - \gamma\, x_2^{(k)}.
\end{array}
\right.
$$

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 14 (Theoretical):** Compute the vector ${\bf{x}}^{(1)}$ obtained after the first iteration, from the initial vector ${\bf{x}}^{(0)}=(1,1,1)^\top$.
</div>

=== BEGIN MARK SCHEME ===

We just have to choose in the previous formulas $k=0$ and
$x_1^{(0)}=x_2^{(0)}=x_3^{(0)}=1$. We get

$$
\left\{
\begin{array}{l}
x_1^{(1)}= 1 - \alpha\, x_3^{(0)} = 1 - \alpha \\[5mm]
x_2^{(1)}= 3 - \beta\, x_1^{(0)}=3 -\beta \\[5mm]
x_3^{(1)}= 2 - \gamma\, x_2^{(0)} =2-\gamma.
\end{array}
\right.
$$

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 15 (Theoretical):** Find a condition on $\alpha$, $\beta$, $\gamma$ which ensures that the Jacobi method converges.
</div>

=== BEGIN MARK SCHEME ===

The iteration matrix for the Jacobi method is given by

$$
  B_J 
  = \Big( \operatorname{diag}(A) \Big)^{-1}\Big( \operatorname{diag}(A)-A\Big) 
  =I-\operatorname{diag}(A)^{-1}A =
                   \begin{pmatrix}
                         0 & 0 & -\alpha \\
                         -\beta & 0 & 0 \\
                         0 & -\gamma & 0
                        \end{pmatrix}.
$$

The eigenvalues of $B_J$ are given by

$$
            \det \begin{pmatrix}
                         \lambda & 0 & \alpha \\
                         \beta & \lambda & 0 \\
                         0 & \gamma & \lambda
                        \end{pmatrix} =
            \lambda^3 + \alpha\beta\gamma = 0 \quad \Longrightarrow \quad 
            \lambda = (-\alpha\beta\gamma)^{\frac{1}{3}}.
$$

The method converges if and only if $\rho(B_J) < 1$, so we require $|\alpha\beta\gamma| < 1$.

=== END MARK SCHEME ===

<hr style="clear:both">

## The end

Outstanding! You've reached the end of the ninth exercise notebook. A long one.